In [8]:
import pandas as pd
import numpy as np



In [ ]:
!pip install s3fs

In [11]:
def preprocess_data(file_path):
    try:
        # Specify the delimiter and decimal handling for non-standard formats
        df = pd.read_csv(
            file_path,
            delimiter=";",  # Use semicolon as the delimiter
            decimal=",",    # Handle commas as decimal points
            encoding="utf-8"  # Ensure proper encoding
        )
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return None

    try:
        # Convert to datetime columns
        df["firstorder"] = pd.to_datetime(df["firstorder"], format="%d.%m.%y", errors="coerce")
        df["lastorder"] = pd.to_datetime(df["lastorder"], format="%d.%m.%y", errors="coerce")
        df["created"] = pd.to_datetime(df["created"], format="%d.%m.%y", errors="coerce")

        # Drop Rows with null values
        df = df.dropna()

        # Create Column which gives the days between 
        # the last order and the first order
        df["first_last_days_diff"] = (df["lastorder"] - df["firstorder"]).dt.days

        # Create Column which gives the days between 
        # when the customer record was created and the first order
        df["created_first_days_diff"] = (df["created"] - df["firstorder"]).dt.days

        # Drop Columns
        df.drop(["custid", "created", "firstorder", "lastorder"], axis=1, inplace=True)

        # Apply one-hot encoding on favday and city columns
        df = pd.get_dummies(df, prefix=["favday", "city"], columns=["favday", "city"])
        return df
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None


# Set the required configurations
model_name = "churn_model"
env = "dev"

# S3 Bucket
default_bucket = "customer-chunk-genai-project"

# Preprocess the dataset
storedata = preprocess_data(f"s3://{default_bucket}/data/storedata_total.csv")

In [12]:
storedata.head()

,retained,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,first_last_days_diff,...,favday_Monday,favday_Saturday,favday_Sunday,favday_Thursday,favday_Tuesday,favday_Wednesday,city_BLR,city_BOM,city_DEL,city_MAA
0,0,29,100.000000,3.448276,14.52,0.000000,0,0,0,0,...,True,False,False,False,False,False,False,False,True,False
1,1,95,92.631579,10.526316,83.69,0.181641,1,1,1,1024,...,False,False,False,False,False,False,False,False,True,False
2,0,0,0.000000,0.000000,33.58,0.059908,0,0,0,217,...,False,False,False,False,False,True,False,False,True,False
3,0,0,0.000000,0.000000,54.96,0.000000,0,0,0,0,...,False,False,False,True,False,False,False,True,False,False
4,1,30,90.000000,13.333333,111.91,0.008850,0,0,0,791,...,True,False,False,False,False,False,False,True,False,False


In [13]:
print(storedata.columns)

Index(['retained', 'esent', 'eopenrate', 'eclickrate', 'avgorder', 'ordfreq',
       'paperless', 'refill', 'doorstep', 'first_last_days_diff',
       'created_first_days_diff', 'favday_Friday', 'favday_Monday',
       'favday_Saturday', 'favday_Sunday', 'favday_Thursday', 'favday_Tuesday',
       'favday_Wednesday', 'city_BLR', 'city_BOM', 'city_DEL', 'city_MAA'],
      dtype='object')


In [14]:
def split_datasets(df):
    y = df.pop("retained")
    X_pre = df
    y_pre = y.to_numpy().reshape(len(y), 1)
    feature_names = list(X_pre.columns)
    X = np.concatenate((y_pre, X_pre), axis=1)
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(.7*len(X)), int(.85*len(X))])
    return feature_names, train, validation, test


# Split dataset
feature_names, train, validation, test = split_datasets(storedata)

# Save datasets in Amazon S3
pd.DataFrame(train).to_csv(f"s3://{default_bucket}/data/train/train.csv",
                           header=False, index=False)
pd.DataFrame(validation).to_csv(f"s3://{default_bucket}/data/validation/validation.csv",
                                header=False, index=False)
pd.DataFrame(test).to_csv(f"s3://{default_bucket}/data/test/test.csv",
                          header=False, index=False)